<a href="https://colab.research.google.com/github/Sushobhan55/Algo/blob/master/Efficient%20Frontier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This program optimizes a stock portfolio using the Efficient Frontier

In [ ]:
# import the required libraries
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import datetime as dt
import seaborn as sns

In [ ]:
#get the start and end dates for the stocks for backtesting
start = dt.datetime(2016,1,1)
end = dt.datetime.now()

In [ ]:
#get the tickers in the portfolio
tickers = ['AAPL','AMZN','GM','JNJ','NVDA','TXN','TSLA','SWKS','MS','ETSY','CLX','NFLX','LUV','XOM','TSM','GOOGL','DIS','JPM','PG','TDOC']

In [ ]:
#assign weights to the stocks
weights = np.array(20*[0.05])

In [ ]:
#create a dataframe to store the adjusted close price of the stocks
my_portfolio = pd.DataFrame()

In [ ]:
#store the adjusted close price of the stock into the dataframe
for ticker in tickers:
  my_portfolio[ticker]=web.DataReader(ticker,'yahoo',start,end)['Adj Close']

In [ ]:
my_portfolio.head()

In [ ]:
#visualize the portfolio
sns.set_style('whitegrid')
chart= sns.lineplot(data=my_portfolio)
chart.set_title('Price history of stocks in My Portfolio')
chart.set_ylabel('Adj Close in USD')

In [ ]:
#store the daily returns on individual stocks
returns = my_portfolio.pct_change()
returns = returns.dropna()
returns

In [ ]:
#create the annualized variance-covariance matrix
annual_varcov_matrix = returns.cov()*252
annual_varcov_matrix

In [ ]:
#calculate the portfolio variance
portfolio_var = np.dot(weights.T, np.dot(annual_varcov_matrix, weights))
portfolio_var

0.04676055267067325

In [ ]:
#calculate the portfolio volatility/standard deviation
portfolio_vol = np.sqrt(portfolio_var)
portfolio_vol

0.21624188463540833

In [ ]:
#calcualte the annual portfolio return
annual_potfolio_return = np.sum(returns.mean()*weights) * 252
annual_potfolio_return

0.3367423961017082

In [ ]:
#show the expected return and risk(volatility) 
print('Expected annual return: ',round(annual_potfolio_return,2)*100,'%')
print('Risk/Annual volatility: ',round(portfolio_vol,2)*100,'%')

Expected annual return:  34.0 %
Risk/Annual volatility:  22.0 %


In [ ]:
#we need higher return for a lower risk
#now we optimize the portfolio by using a library PyPortfolioOpt

In [ ]:
pip install PyPortfolioOpt

In [ ]:
#import the libraries for portfolio optimization 
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import objective_functions

In [ ]:
#calculate the expected return and 
#the annualized sample covariance matrix of individual returns
mu = expected_returns.mean_historical_return(my_portfolio)
S = risk_models.sample_cov(my_portfolio)

In [ ]:
#optimize for maximum sharpe ratio
ef = EfficientFrontier(mu,S)
ef.add_objective(objective_functions.L2_reg, gamma=2)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.portfolio_performance(verbose = True)
print(cleaned_weights)

Expected annual return: 59.1%
Annual volatility: 28.3%
Sharpe Ratio: 2.02
OrderedDict([('AAPL', 0.0613), ('AMZN', 0.05755), ('GM', 0.00706), ('JNJ', 0.01615), ('NVDA', 0.13686), ('TXN', 0.04026), ('TSLA', 0.12862), ('SWKS', 0.01127), ('MS', 0.02175), ('ETSY', 0.16372), ('CLX', 0.01416), ('NFLX', 0.05125), ('LUV', 0.0), ('XOM', 0.0), ('TSM', 0.08699), ('GOOGL', 0.02333), ('DIS', 0.01152), ('JPM', 0.02609), ('PG', 0.01762), ('TDOC', 0.12447)])


/usr/local/lib/python3.6/dist-packages/pypfopt/efficient_frontier.py:248: UserWarning: max_sharpe transforms the optimisation problem so additional objectives may not work as expected.
  "max_sharpe transforms the optimisation problem so additional objectives may not work as expected."


In [ ]:
#get the discrete allocation of each share per stock
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
latest_prices = get_latest_prices(my_portfolio)
weights=cleaned_weights
da = DiscreteAllocation(weights,latest_prices, total_portfolio_value=1000000)
allocation , leftover = da.lp_portfolio()
print('Discrete allocation: ', allocation)
print('Funds remaining: ${:.2f}'.format(leftover))

Discrete allocation:  {'AAPL': 460, 'AMZN': 18, 'GM': 132, 'JNJ': 97, 'NVDA': 223, 'TXN': 222, 'TSLA': 161, 'SWKS': 58, 'MS': 286, 'ETSY': 717, 'CLX': 77, 'NFLX': 92, 'TSM': 621, 'GOOGL': 11, 'DIS': 62, 'JPM': 180, 'PG': 137, 'TDOC': 427}
Funds remaining: $2.28
